# AI Investor Project

## Step 1: Initial Analysis and User Inputs

1. Specify Today
2. Specify Tomorrow
3. Specify Aggressive vs. Conservative vs. Moderate
4. Specify how many stock you want to view
5. Allow the models to run (8 Minutes)
6. Receive Results

In [15]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

np.random.seed(1234)
period = 6
date_split = '2021-04-29'
build_end_date = '2021-09-30'

## User Inputs
today = input("What is today's date: valid through 2021-10-10?")
tomorrow = input("What is tomorrow's date: valid through 2021-10-11")
nbr = int(input("How many stocks do you want to see?"))
strategy = input("What is your strategy? (aggresive, conservative, moderate)")

print("Analyzing. . . . .")
print("Come back in 10 minutes. . . .")

## Import The Ticker Reference Data
ticker_data = pd.read_csv('constituents.csv')
ticker_to_remove = ['IVZ', 'CARR', 'CTVA', 'DOW', 'FOX', 'FOXA', 'OGN', 'OTIS']
ticker_data = ticker_data[~ticker_data['Symbol'].isin(ticker_to_remove)]
tickers = ticker_data['Symbol']

## Necessary Functions

def closing_price_lag(index, close_prices, period = 6, column_prefix = 't_minus'):
    X = []
    ind = []
    cols = []
    
    for i in range(len(close_prices)-period):
        X.append(close_prices[i:i+period])
        ind.append(index[i+period])
    
    for i in range(period,0,-1):
        words = [column_prefix, str(i)]
        cols.append("".join(words))
        
    return X,ind,cols


## Build the model for each ticker
#### Empty Lists to fill for the For Loop
ticker_predictions = []
model_predictions = []
date_predictions = []
todays_close = []

ticker_std = []
close_std = []

#### The Almight For Loop
for ticker in tickers:
    
    ## Importing Data
    data = yf.download(ticker, start='2019-01-01', end = '2021-10-15', progress=False)
    data.columns = ['open', 'high', 'low', 'close', 'adj_close', 'volume']
    data = data[~data['close'].isnull()]
    
    ## Feature Engineering
    
    #### 1) Lagged Closing Prices
    X,ind,cols = closing_price_lag(index = list(np.array(data.index)), close_prices = list(data['close']), period = period)
    
    #### 2) Lagged High / Low Prices
    data_sub = data[['high', 'low']]
    data_sub['t_minus1_high'] = data_sub['high'].shift(1)
    data_sub['t_minus1_low'] = data_sub['low'].shift(1)
    high_low_lags = data_sub[['t_minus1_high', 't_minus1_low']]

    #### 3) Putting Steps 1 & 2 Together
    test_features = pd.DataFrame(X, index = ind, columns = cols)
    full_data = test_features.merge(data, how = 'inner', left_index=True, right_index=True)
    full_data = full_data.merge(high_low_lags, how = 'inner', left_index=True, right_index=True)

    #### 4) Previous Day - Intraday Volatility
    full_data['prev_day_intraday_volatility'] = abs(full_data['t_minus1_high']-full_data['t_minus1_low'])/(full_data['t_minus1'])

    ## Modeling Building
    
    #### 1) Training / Testing Data Split
    ####### Columns to Remove
    non_x_columns = ['open', 'high', 'low', 'close', 'adj_close', 'volume']
    training_data = full_data.loc[:build_end_date]
    y_train = training_data['close'].tolist()
    X_train = training_data.drop(non_x_columns, axis=1).values.tolist()
    
    #### 2) Final Model Build
    reg = LinearRegression(normalize=True).fit(X_train, y_train)
    
    #### 3) Predictions
    todays_data = [full_data.loc[today].drop(non_x_columns).tolist()]
    tomorrow_pred = reg.predict(todays_data)
    
    ticker_predictions.append(ticker)
    model_predictions.append(tomorrow_pred)
    date_predictions.append(tomorrow)
    
    ## Conservative vs. Aggressive Strategy (6month STD)
    close_std.append(data['close'].loc['2021-05-01':'2021-10-29'].std())
    ticker_std.append(ticker)
    
    ## Append Today's Close
    todays_close.append(full_data.loc[today].close)


## Turn the created Lists into DataFrames
model_predictions = {
    'ticker' : ticker_predictions,
    'prediction': np.concatenate(model_predictions).ravel(),
    'todays_close': todays_close
}

model_prediction = pd.DataFrame(model_predictions, index=None)

std = {
    'ticker' : ticker_std,
    'std_close' : close_std
}

std_close = pd.DataFrame(std, index=None)

## Calculate Expected Percent Change
model_prediction['perc_change'] = (model_prediction['prediction'] / model_prediction['todays_close']) - 1

## Join all the tables together
all_model_data = model_prediction.merge(std_close, how = 'left', on = 'ticker')
all_model_data = all_model_data.merge(ticker_data, how = 'left', left_on = 'ticker', right_on = 'Symbol')

## Create the Strategy Column
conditions = [
    all_model_data['std_close'] < 2.5,
    all_model_data['std_close'] < 8,
    all_model_data['std_close'] >= 8
    ]

values = ['conservative', 'moderate', 'aggressive']

all_model_data['strategy'] = np.select(conditions, values)

## Create the view and print the data
columns_to_display = ['Name', 'ticker','perc_change', 'todays_close', 'prediction']

print(" ")
print(" ")
print("##########  Here are your results!  ##########")
display(all_model_data[(all_model_data['strategy']==strategy) & (all_model_data['perc_change']>0)][columns_to_display].sort_values('perc_change', ascending =False).head(n=nbr))
print(" ")
print(" ")

## While Loop to Cycle Through Different Views
end = int(input("Would you like to continue? (1 for Yes, 2 to Exit)"))

while end == 1:
    strategy = input("Would you like to change your strategy? (conservative, moderate, aggressive)")
    nbr = int(input("How many stocks would you like to see?"))
    print(" ")
    print(" ")
    print("##########  Updated Results  ##########")
    display(all_model_data[(all_model_data['strategy']==strategy) & (all_model_data['perc_change']>0)][columns_to_display].sort_values('perc_change', ascending =False).head(n=nbr))
    print(" ")
    end = int(input("Would you like to continue? (1 for Yes, 2 to Exit)"))


What is today's date: valid through 2021-10-10?2021-10-05
What is tomorrow's date: valid through 2021-10-112021-10-06
How many stocks do you want to see?5
What is your strategy? (aggresive, conservative, moderate)moderate
Analyzing. . . . .
Come back in 10 minutes. . . .
##########  Here are your results!  ##########


,Name,ticker,perc_change,todays_close,prediction
351,Penn National Gaming,PENN,0.037764,72.519997,75.258658
477,Welltower,WELL,0.028220,81.739998,84.046711
462,VF Corporation,VFC,0.023406,67.459999,69.038996
239,Incyte,INCY,0.023128,65.330002,66.840964
243,International Flavors & Fragrances,IFF,0.020481,135.470001,138.244598


 
Would you like to continue? (1 for Yes, 2 to Exit)1
Would you like to change your strategy? (conservative, moderate, aggressive)aggressive
How many stocks would you like to see?10
##########  Updated Results  ##########


,Name,ticker,perc_change,todays_close,prediction
170,Equinix,EQIX,0.033077,760.070007,785.210494
81,Caesars Entertainment,CZR,0.032609,114.760002,118.502227
308,Moderna,MRNA,0.022745,332.109985,339.663695
18,Alexandria Real Estate Equities,ARE,0.012203,192.130005,194.474639
272,Lamb Weston,LW,0.012193,62.450001,63.211433
71,Booking Holdings,BKNG,0.011474,2432.340088,2460.248621
384,Regeneron Pharmaceuticals,REGN,0.011056,550.900024,556.990575
188,Fidelity National Information Services,FIS,0.010436,120.339996,121.595807
377,Quanta Services,PWR,0.010410,112.660004,113.832787
179,Expedia Group,EXPE,0.009864,169.100006,170.767946


 
Would you like to continue? (1 for Yes, 2 to Exit)1
Would you like to change your strategy? (conservative, moderate, aggressive)conservative
How many stocks would you like to see?5
##########  Updated Results  ##########


,Name,ticker,perc_change,todays_close,prediction
457,Ventas,VTR,0.043358,54.200001,56.550028
215,Hanesbrands,HBI,0.032186,16.930000,17.474907
300,Merck & Co.,MRK,0.019885,81.599998,83.222655
218,Healthpeak Properties,PEAK,0.017120,33.549999,34.124382
264,Kimco Realty,KIM,0.016322,21.480000,21.830586


 
Would you like to continue? (1 for Yes, 2 to Exit)1
Would you like to change your strategy? (conservative, moderate, aggressive)conservative
How many stocks would you like to see?5
##########  Updated Results  ##########


,Name,ticker,perc_change,todays_close,prediction
457,Ventas,VTR,0.043358,54.200001,56.550028
215,Hanesbrands,HBI,0.032186,16.930000,17.474907
300,Merck & Co.,MRK,0.019885,81.599998,83.222655
218,Healthpeak Properties,PEAK,0.017120,33.549999,34.124382
264,Kimco Realty,KIM,0.016322,21.480000,21.830586


 
Would you like to continue? (1 for Yes, 2 to Exit)2


## Step 2: Use to revisit the analysis from Step 1. 

In [16]:
end = int(input("Would you like to revisit the analysis? (1 for Yes, 2 to Exit)"))

while end == 1:
    strategy = input("Would you like to change your strategy? (conservative, moderate, aggressive)")
    nbr = int(input("How many stocks would you like to see?"))
    print(" ")
    print(" ")
    print("##########  Updated Results  ##########")
    display(all_model_data[(all_model_data['strategy']==strategy) & (all_model_data['perc_change']>0)][columns_to_display].sort_values('perc_change', ascending =False).head(n=nbr))
    print(" ")
    end = int(input("Would you like to continue? (1 for Yes, 2 to Exit)"))


Would you like to revisit the analysis? (1 for Yes, 2 to Exit)1
Would you like to change your strategy? (conservative, moderate, aggressive)moderate
How many stocks would you like to see?10
 
 
##########  Updated Results  ##########


,Name,ticker,perc_change,todays_close,prediction
351,Penn National Gaming,PENN,0.037764,72.519997,75.258658
477,Welltower,WELL,0.028220,81.739998,84.046711
462,VF Corporation,VFC,0.023406,67.459999,69.038996
239,Incyte,INCY,0.023128,65.330002,66.840964
243,International Flavors & Fragrances,IFF,0.020481,135.470001,138.244598
16,Alaska Air Group,ALK,0.020299,61.599998,62.850387
186,Federal Realty Investment Trust,FRT,0.020058,120.110001,122.519138
200,Freeport-McMoRan,FCX,0.019829,32.200001,32.838499
89,Cboe Global Markets,CBOE,0.019826,120.839996,123.235741
142,Digital Realty Trust,DLR,0.019077,142.699997,145.422354


 
Would you like to continue? (1 for Yes, 2 to Exit)1
Would you like to change your strategy? (conservative, moderate, aggressive)aggressive
How many stocks would you like to see?20
 
 
##########  Updated Results  ##########


,Name,ticker,perc_change,todays_close,prediction
170,Equinix,EQIX,0.033077,760.070007,785.210494
81,Caesars Entertainment,CZR,0.032609,114.760002,118.502227
308,Moderna,MRNA,0.022745,332.109985,339.663695
18,Alexandria Real Estate Equities,ARE,0.012203,192.130005,194.474639
272,Lamb Weston,LW,0.012193,62.450001,63.211433
71,Booking Holdings,BKNG,0.011474,2432.340088,2460.248621
384,Regeneron Pharmaceuticals,REGN,0.011056,550.900024,556.990575
188,Fidelity National Information Services,FIS,0.010436,120.339996,121.595807
377,Quanta Services,PWR,0.010410,112.660004,113.832787
179,Expedia Group,EXPE,0.009864,169.100006,170.767946


 
Would you like to continue? (1 for Yes, 2 to Exit)2
